In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import getpass

In [ ]:
openai_api_key = getpass.getpass(prompt='Openai Api Key')

## 1. Zero-Shot ReAct Agent (ZERO_SHOT_REACT_DESCRIPTION)

In [ ]:
# Define a simple tool
def echo_tool(query):
    return f"You asked: {query}"

dummy_tool = Tool(
    name="EchoTool",
    func=echo_tool,
    description="Repeats whatever input is given."
)
# Initialize OpenAI model
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)

# Create the Zero-Shot ReAct Agent (no tools, just reasoning)
agent = initialize_agent(
    tools=[dummy_tool],  
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent
response = agent.run("What are the three laws of motion?")
print(response)


## 2. OpenAI Functions Agent (OPENAI_FUNCTIONS)

In [ ]:
# Define a simple tool
def get_current_time(*args):
    for arg in args:
        print("argument ",arg)
    from datetime import datetime
    return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

time_tool = Tool(
    name="GetCurrentTime",
    func=get_current_time,
    description="Returns the current time."
)

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# Create an OpenAI Functions Agent with the tool
agent = initialize_agent(
    tools=[time_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# Run the agent
response = agent.run("What time is it?")
print(response)

## 3. ReAct Agent (STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION)

In [ ]:
# Define a tool
from langchain.tools import StructuredTool
from pydantic import BaseModel

# Define input schema
class WikiInput(BaseModel):
    query: str

# Define Wikipedia search function
def search_wikipedia(query: str):
    return f"Simulated Wikipedia search results for: {query}"

# Convert function to a StructuredTool
wiki_tool = StructuredTool.from_function(
    func=search_wikipedia,
    name="WikipediaSearch",
    description="Searches Wikipedia for a given query.",
    args_schema=WikiInput,  # ✅ Enforces structured input
)

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# Create a structured ReAct agent
agent = initialize_agent(
    tools=[wiki_tool],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent
response = agent.run("Tell me about quantum mechanics using Wikipedia.")
print(response)

## 4. Conversational Agent (CONVERSATIONAL_REACT_DESCRIPTION)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# Create a conversational agent
agent = initialize_agent(
    tools=[],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# Run multiple queries to see memory in action
response1 = agent.run("Who was Albert Einstein?")
print(response1)

response2 = agent.run("What was his biggest contribution to physics?")
print(response2)

## 5. Self-Querying Agent (SELF_ASK_WITH_SEARCH)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# ✅ Simulated Search Function (Replace with API calls if needed)
def google_search(query: str):
    search_results = {
        "latest breakthroughs in quantum computing": 
            "IBM's 127-qubit Eagle processor, Google's quantum supremacy, and new quantum algorithms.",
        "top AI companies in 2024": 
            "OpenAI, Google DeepMind, Anthropic, and Meta AI are leading AI research.",
        "climate change solutions": 
            "Renewable energy, carbon capture, and afforestation are key solutions."
    }
    return search_results.get(query.lower(), "No relevant search results found.")

# ✅ Define the Tool (Must be named "Intermediate Answer")
search_tool = Tool(
    name="Intermediate Answer",  # REQUIRED for SELF_ASK_WITH_SEARCH
    func=google_search,
    description="Searches the web for the most relevant and recent information."
)

# ✅ Initialize OpenAI LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key,system_message="Always provide detailed responses in full sentences. Never respond with just 'Yes' or 'No'.")

# ✅ Create the Self-Querying Agent
agent = initialize_agent(
    tools=[search_tool],  # Tool MUST be named "Intermediate Answer"
    llm=llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
    handle_parsing_errors=True
)

# ✅ Run the agent with a complex query
query = "What are the latest breakthroughs in quantum computing?"
response = agent.run(query)

# ✅ Print the response
print("\n🧠 AI Response:", response)


## 6. Plan-and-Execute Agent (PLAN_AND_EXECUTE)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor
from langchain_experimental.plan_and_execute.planners import chat_planner
from langchain_experimental.plan_and_execute.executors import agent_executor
# ✅ Simulated search function
def google_search(query: str):
    search_results = {
        "latest breakthroughs in AI": "OpenAI's GPT-5 research, DeepMind's AlphaFold updates, and advances in multimodal AI.",
        "top space discoveries in 2024": "NASA's James Webb discoveries, exoplanets, and new black hole theories.",
    }
    return search_results.get(query.lower(), "No relevant search results found. Try checking NASA or OpenAI websites.")

# ✅ Define a search tool
search_tool = Tool(
    name="GoogleSearch",
    func=google_search,
    description="Search for the latest scientific and technological advancements."
)

# ✅ Initialize OpenAI LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# ✅ Correct Planner & Executor setup for LangChain 0.3.18
planner = chat_planner(llm=llm)  # Replaces `ChatPlanner`
executor = agent_executor(llm=llm, tools=[search_tool])  # Replaces `AgentExecutor`

# ✅ Create Plan-and-Execute Agent
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

# ✅ Run the agent with a complex query
query = "Find the latest breakthroughs in AI and summarize them."
response = agent.run(query)

# ✅ Print the response
print("\n🧠 AI Response:", response)